## Here we will learn how to use Pipelines to cleanup ML code

#### 1. Import pandas and loading the csv file

In [1]:
import pandas as pd
path="melb_data.csv"
data=pd.read_csv(path)
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [2]:
data.shape

(13580, 21)

In [3]:
#selecting the target
from sklearn.model_selection import train_test_split
y=data.Price
X=data.drop(['Price'], axis=1)
#spliting the data into trainig and validation set
X_train,X_valid,y_train,y_valid=train_test_split(X,y,train_size=0.8, test_size=0.2)

In [4]:
#splitting the numerical columns from the trainig set
num_cols=([col for col in X_train
          if X_train[col].dtype in ['int','float']])
num_cols

['Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [5]:
cat_cols=([col for col in X_train
          if X_train[col].dtype =='object'])
cat_cols

['Suburb',
 'Address',
 'Type',
 'Method',
 'SellerG',
 'Date',
 'CouncilArea',
 'Regionname']

In [6]:
my_cols=num_cols +cat_cols


In [7]:
#now preprocessing the data
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
num_transformer=SimpleImputer(strategy='constant')
cat_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('OneHot', OneHotEncoder(handle_unknown='ignore'))
])

In [8]:
Preprocessor=ColumnTransformer(transformers=[
    ('num',num_transformer,num_cols),
    ('cat',cat_transformer, cat_cols)
])

In [9]:
#definig the model
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(random_state=0)

In [ ]:
my_Pipeline=Pipeline(steps=[
    ('preprocessor',Preprocessor),
    ('model',model)
])
X_train=X_train[my_cols].copy()
X_valid=X_valid[my_cols].copy()
my_Pipeline.fit(X_train,y_train)

In [ ]:
my_Pipeline.fit(X_train,y_train)


In [ ]:
preds=my_Pipeline.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae= mean_absolute_error(y_valid,preds)
mae